In [1]:
import rasterio as rio
import numpy as np
import logging
import sys
from datetime import datetime

In [2]:
# Investigate if it worked

logging.basicConfig(stream=sys.stderr, level=logging.INFO)

merged_cc_data = "merged_cc_data.tif"
with rio.open(merged_cc_data, "r") as src:
    profile = src.profile
    logging.info(profile)
    
    windows = src.block_windows()
    
    curr_row = None
    
    for win_ix, window in windows:
        data = src.read(indexes=1, window=window)
        
        if(curr_row != win_ix[0]):
            curr_row = win_ix[0]
            logging.info("Processing row:" + str(curr_row))
                
        if np.sum(data) > 0:
            logging.info(np.sum(data))

INFO:root:{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 432360, 'height': 140517, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.000832639467111, 0.0, -180.0,
       0.0, -0.000832639467111, 61.0), 'blockxsize': 256, 'blockysize': 256, 'compress': 'lzw', 'interleave': 'band', 'tiled': True}
INFO:root:Processing row:0
INFO:root:Processing row:1
INFO:root:Processing row:2
INFO:root:Processing row:3
INFO:root:Processing row:4
INFO:root:Processing row:5
INFO:root:Processing row:6
INFO:root:Processing row:7
INFO:root:Processing row:8
INFO:root:Processing row:9
INFO:root:Processing row:10
INFO:root:Processing row:11
INFO:root:Processing row:12
INFO:root:Processing row:13
INFO:root:Processing row:14
INFO:root:Processing row:15
INFO:root:Processing row:16
INFO:root:Processing row:17
INFO:root:Processing row:18
INFO:root:Processing row:19
INFO:root:Processing row:20
INFO:root:Processing row:21
INFO:root:Processing row:22
INFO:root:Processing row:23
INFO:roo

KeyboardInterrupt: 

In [1]:
import rasterio as rio
import numpy as np
from datetime import datetime
import logging
import sys

logging.basicConfig(stream=sys.stderr, level=logging.INFO)
# If true, this function will print out the sum of the values being input into merged_cc_data
VERBOSE=False

merged_cc_data = "merged_cc_data_ft.tif"

with rio.open("https://wri-public-data.s3.amazonaws.com/resourcewatch/raster/cli_024_sea_level_rise/cli_024_sea_level_rise_ft.tif", "r") as src:
    profile = src.profile
    logging.info(profile)
    
profile["count"] = 1

with rio.open(merged_cc_data, "w", **profile) as dst:
    # Iterate over the input data, updating the merged file as we go
    # Do this for every block in the larger block_windows
    
    with rio.open("https://wri-public-data.s3.amazonaws.com/resourcewatch/raster/cli_024_sea_level_rise/cli_024_sea_level_rise_ft.tif", "r") as src:
        profile = src.profile
        
        num_bands = profile["count"]
        blockx = profile["blockxsize"]
        blocky = profile["blockysize"]
        
        height = profile["height"]
        width = profile["width"]
        
        num_rows = int(height/blocky)
        num_cols = int(width/blockx)
        
        logging.info("num_rows: " + str(num_rows))
        logging.info("num_cols: " + str(num_cols))
        
        dtype = profile["dtype"]
        
        windows = src.block_windows()
        
        curr_row = None
        start_time = datetime.now()
        time_step = datetime.now()
        
        for win_ix, window in windows:
            
            if(curr_row != win_ix[0]):
                curr_row = win_ix[0]
                curr_time = datetime.now()
                logging.info("Processing Row: " + str(curr_row))
                logging.info("Time for row: " + str(curr_time-time_step))
                logging.info("Total time elapsed: " + str(curr_time-start_time))
                time_step = curr_time
    
            x = window[0][1] - window[0][0]
            y = window[1][1] - window[1][0]
            merged_data = np.zeros((x,y), dtype=dtype)
            
            for bnd in range(num_bands, 0, -1):
                data = src.read(indexes=bnd, window=window)
                merged_data[data==1] = bnd
            
            sum_data = np.sum(merged_data)
            
            if (sum_data > 0) & VERBOSE:
                logging.info("Sum of merged_data: " + str(sum_data))
            dst.write(merged_data, indexes=1, window=window)

INFO:root:{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 432360, 'height': 140517, 'count': 10, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.000832639467111, 0.0, -180.0,
       0.0, -0.000832639467111, 61.0), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}
INFO:root:num_rows: 548
INFO:root:num_cols: 1688
INFO:root:Processing Row: 0
INFO:root:Time for row: 0:00:00.000018
INFO:root:Total time elapsed: 0:00:00.000027


KeyboardInterrupt: 

In [ ]:
import rasterio as rio
import numpy as np
from datetime import datetime
import logging

logging.basicConfig(stream=sys.stderr, level=logging.INFO)

merged_cc_data = "merged_cc_data_meters.tif"

conversions_of_band_to_meters = {
    1:.5,
    2:1.,
    3:1.5,
    4:2.,
    5:2.5,
    6:3.,
    7:5.,
    8:10.,
    9:20.,
    10:30.
}

with rio.open("https://wri-public-data.s3.amazonaws.com/resourcewatch/raster/cli_024_sea_level_rise/cli_024_sea_level_rise_m.tif", "r") as src:
    profile = src.profile
    print(profile)
    
profile["count"] = 1

with rio.open(merged_cc_data, "w", **profile) as dst:
    # Iterate over the input data, updating the merged file as we go
    # Do this for every block in the larger block_windows
    
    with rio.open("https://wri-public-data.s3.amazonaws.com/resourcewatch/raster/cli_024_sea_level_rise/cli_024_sea_level_rise_m.tif", "r") as src:
        profile = src.profile
        
        num_bands = profile["count"]
        blockx = profile["blockxsize"]
        blocky = profile["blockysize"]
        
        height = profile["height"]
        width = profile["width"]
        
        num_rows = int(height/blocky)
        num_cols = int(width/blockx)
        
        logging.info("num_rows: " + str(num_rows))
        logging.info("num_cols: " + str(num_cols))
        
        dtype = profile["dtype"]
        
        windows = src.block_windows()
        
        curr_row = None
        start_time = datetime.now()
        time_step = datetime.now()
        
        for win_ix, window in windows:
            
            if(curr_row != win_ix[0]):
                curr_row = win_ix[0]
                curr_time = datetime.now()
                logging.info("Processing Row: " + str(curr_row))
                logging.info("Time for row: " + str(curr_time-time_step))
                logging.info("Total time elapsed: " + str(curr_time-start_time))
                time_step = curr_time
    
            x = window[0][1] - window[0][0]
            y = window[1][1] - window[1][0]
            merged_data = np.zeros((x,y), dtype=dtype)
            
            for bnd in range(num_bands, 0, -1):
                data = src.read(indexes=bnd, window=window)
                merged_data[data==1] = conversions_of_band_to_meters[bnd]

            dst.write(merged_data, indexes=1, window=window)

In [34]:
from rasterio.crs import CRS
from affine import Affine

sample_tif = "climate_central_merge_sample.tif"
profile = {
    'driver': 'GTiff', 
   'dtype': 'uint8', 
   'nodata': 0.0, 
   'width': 256, 
   'height': 256, 
   'count': 10, 
   'crs': CRS({'init': 'epsg:4326'}), 
   'transform': Affine(0.000832639467111, 0.0, -180.0,
0.0, -0.000832639467111, 61.0), 
   'blockxsize': 256, 
   'blockysize': 256, 
   'tiled': True, 
   'compress': 'lzw', 
   'interleave': 'pixel'
}

num_bands = profile["count"]

with rio.open(sample_tif, "w", **profile) as dst:
    # Make data that conforms to the CC style... 
    # i.e. if an earlier band has a 1 in a cell, the later ones do too
    
    new_data = np.zeros((256, 256), dtype="uint8")
    
    for bnd in range(1,num_bands+1):
        print("Writing band", bnd)
        
        # How to randomly select in shape of 2d array:
        # https://www.reddit.com/r/learnpython/comments/2ol1n4/taking_random_sample_from_a_2d_numpy_array/
        new_ones = np.random.choice([True,False], new_data.shape)
        new_data[new_ones] = 1
        dst.write(new_data, indexes=bnd)

Writing band 1
Writing band 2
Writing band 3
Writing band 4
Writing band 5
Writing band 6
Writing band 7
Writing band 8
Writing band 9
Writing band 10


In [36]:
with rio.open(sample_tif, "r") as src:
        profile = src.profile
        num_bands = profile["count"]
        for bnd in range(num_bands, 0, -1):
            print("Copying data from band", bnd)
            data = src.read(indexes=bnd)
            merged_data[data==1] = bnd

Copying data from band 10
Copying data from band 9
Copying data from band 8
Copying data from band 7
Copying data from band 6
Copying data from band 5
Copying data from band 4
Copying data from band 3
Copying data from band 2
Copying data from band 1


In [42]:
sample_merged_tif = "climate_central_merged_sample.tif"
profile = {
    'driver': 'GTiff', 
   'dtype': 'uint8', 
   'nodata': 0.0, 
   'width': 256, 
   'height': 256, 
   'count': 1, 
   'crs': CRS({'init': 'epsg:4326'}), 
   'transform': Affine(0.000832639467111, 0.0, -180.0,
0.0, -0.000832639467111, 61.0), 
   'blockxsize': 256, 
   'blockysize': 256, 
   'tiled': True, 
   'compress': 'lzw', 
   'interleave': 'pixel'
}

# Open the file to contain the merged data
with rio.open(sample_merged_tif, "w", **profile) as dst:
    # Iterate over the input data, updating the merged file as we go
    # Would do this for every block in the larger block_windows
    
    merged_data = np.zeros((256,256), dtype="uint8")
    
    with rio.open(sample_tif, "r") as src:
        profile = src.profile
        num_bands = profile["count"]
        for bnd in range(num_bands, 0, -1):
            print("Copying data from band", bnd)
            data = src.read(indexes=bnd)
            merged_data[data==1] = bnd
    
    print(merged_data.shape)
    print(merged_data)
    dst.write(merged_data, indexes=1)

Copying data from band 10
Copying data from band 9
Copying data from band 8
Copying data from band 7
Copying data from band 6
Copying data from band 5
Copying data from band 4
Copying data from band 3
Copying data from band 2
Copying data from band 1
(256, 256)
[[5 1 1 ..., 5 4 3]
 [1 1 5 ..., 4 5 1]
 [1 1 4 ..., 3 5 1]
 ..., 
 [1 1 5 ..., 3 3 1]
 [9 1 3 ..., 2 2 1]
 [2 6 3 ..., 3 1 1]]


In [43]:
with rio.open(sample_merged_tif, "r", **profile) as src:
    merged_profile = src.profile
    print(merged_profile)
    data = src.read(indexes=1)
    
print(data)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 256, 'height': 256, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.000832639467111, 0.0, -180.0,
       0.0, -0.000832639467111, 61.0), 'blockxsize': '256', 'blockysize': '256', 'compress': 'lzw', 'interleave': 'band', 'tiled': False}
[[5 1 1 ..., 5 4 3]
 [1 1 5 ..., 4 5 1]
 [1 1 4 ..., 3 5 1]
 ..., 
 [1 1 5 ..., 3 3 1]
 [9 1 3 ..., 2 2 1]
 [2 6 3 ..., 3 1 1]]


Experiment with multiprocessing

In [ ]:
# Concurrency with rasterio - couldn't figure out how to get this to work
# https://mapbox.github.io/rasterio/topics/concurrency.html
# The compute function mentioned here: https://github.com/mapbox/rasterio/blob/master/rasterio/_example.pyx
# ThreadPoolExecutor: https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.ThreadPoolExecutor

import rasterio as rio
import numpy as np
from rasterio._example import compute

import concurrent.futures
import multiprocessing
import time
max_works = multiprocessing.cpu_count()

merged_cc_data = "merged_cc_data.tif"

with rio.open("https://wri-public-data.s3.amazonaws.com/resourcewatch/raster/cli_024_sea_level_rise/cli_024_sea_level_rise_ft.tif", "r") as src:
    profile = src.profile
    print(profile)
    
profile["count"] = 1

with rio.open(merged_cc_data, "w", **profile) as dst:
    # Iterate over the input data, updating the merged file as we go
    # Do this for every block in the larger block_windows
    
    with rio.open("https://wri-public-data.s3.amazonaws.com/resourcewatch/raster/cli_024_sea_level_rise/cli_024_sea_level_rise_ft.tif", "r") as src:
        profile = src.profile
        
        num_bands = profile["count"]
        blockx = profile["blockxsize"]
        blocky = profile["blockysize"]
        
        height = profile["height"]
        width = profile["width"]
        
        num_rows = int(height/blocky)
        num_cols = int(width/blockx)
        
        print("num_rows:", num_rows)
        print("num_cols:", num_cols)
        
        dtype = profile["dtype"]
        
        def jobs():
            for ij, window in dst.block_windows():
                x = window[0][1] - window[0][0]
                y = window[1][1] - window[1][0]
                merged_data = np.zeros((x,y), dtype=dtype)

                for bnd in range(num_bands, 0, -1):
                    data = src.read(indexes=bnd, window=window)
                    merged_data[data==1] = bnd

                res = np.zeros((x,y), dtype=dtype)
                yield merged_data, res, window, ij
        
        # Submit the jobs to the thread pool executor.
        with concurrent.futures.ThreadPoolExecutor(
                max_workers=multiprocessing.cpu_count()) as executor:

            # Map the futures returned from executor.submit()
            # to their destination windows.
            #
            # The _example.compute function modifies no Python
            # objects and releases the GIL. It can execute
            # concurrently.
            future_to_window = {
                executor.submit(compute, data, res): (data, res, window, ij)
                for data, res, window, ij in jobs()}

            # As the processing jobs are completed, get the
            # results and write the data to the appropriate
            # destination window.
            for future in concurrent.futures.as_completed(
                    future_to_window):

                data, result, window, ij = future_to_window[future]

                print("Computing for index:", ij)
                
                dst.write(result, indexes=1, window=window)